# Day 12: Passage Pathing

In [1]:
from pathlib import Path
from itertools import permutations
from more_itertools import flatten

from aoc2021.util import read_as_list

## Puzzle input data

In [2]:
parse_input = lambda line: tuple(line.rstrip().split('-'))

# Test data.
tdata = list(map(parse_input, [
    'start-A',
    'start-b',
    'A-c',
    'A-b',
    'b-d',
    'A-end',
    'b-end',
]))

# Input data.
data = read_as_list(Path('./day12-input.txt'), func=parse_input)
data[:5]

[('pq', 'GX'), ('GX', 'ah'), ('mj', 'PI'), ('ey', 'start'), ('end', 'PI')]

## Puzzle answers
### Part 1

In [3]:
Input = list[tuple[str,str]]
Graph = dict[str,set[str]]


def expand_graph(data: Input) -> Graph:
    nodes = set(flatten(data))
    edges = list(flatten(map(permutations, data)))
    return {n: set(end for start,end in edges if start == n) for n in nodes}


def is_small(cave: str) -> bool:
    return cave[0].islower()


def drop_node(node: str, graph: Graph) -> Graph:
    return {n: sbns.difference([node]) for n,sbns in graph.items()}


def paths_between(graph: Graph, start: str, end: str) -> list[str]:
    if start == end:
        return [end]
    if is_small(start):
        graph = drop_node(start, graph)
    return [f'{start},{p}' for sbn in graph[start] for p in paths_between(graph, sbn, end) if graph[sbn]]


def cave_paths(data: Input) -> list[str]:
    return paths_between(expand_graph(data), 'start', 'end')


assert set(expand_graph(tdata).keys()) == {'start','A','b','c','d','end'}
assert expand_graph(tdata)['A'] == {'start','b','c','end'}
assert list(map(is_small, ['A','b','CC','dd'])) == [False,True,False,True]
assert len(cave_paths(tdata)) == 10
assert set(cave_paths(tdata)) == {
    'start,A,b,A,c,A,end',
    'start,A,b,A,end',
    'start,A,b,end',
    'start,A,c,A,b,A,end',
    'start,A,c,A,b,end',
    'start,A,c,A,end',
    'start,A,end',
    'start,b,A,c,A,end',
    'start,b,A,end',
    'start,b,end',
}

In [4]:
n = len(cave_paths(data))
print(f'Number of paths through cave system that visit small caves at most once: {n}')

Number of paths through cave system that visit small caves at most once: 5333


### Part 2

In [5]:
def paths_between(graph: Graph, start: str, end: str, repeated: bool = False) -> list[str]:
    if start == end:
        return [end]
    graphs = [graph]
    repeats = [repeated]
    if is_small(start):
        if repeated or start == 'start':
            graphs = [drop_node(start, graph)]
        else:
            repeats = [True, False]
            graphs = [graph, drop_node(start, graph)]
    return set(f'{start},{p}' for rep,g in zip(repeats, graphs) for sbn in g[start] for p in paths_between(g, sbn, end, rep) if g[sbn])


def cave_paths(data: Input) -> list[str]:
    return paths_between(expand_graph(data), 'start', 'end')


assert len(cave_paths(tdata)) == 36

In [6]:
n = len(cave_paths(data))
print(f'Number of paths through cave system that visit at most once all small caves but one (at most twice): {n}')

Number of paths through cave system that visit at most once all small caves but one (at most twice): 146553
